In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* y: unit price_ping
* train : test = 0.7 : 0.3

# Dataset

* MRT: 整合 "MRT_1km","MRT_500m","MRT_300m"
* Park: 整合 "park_500m","park_300m"
* 已包含 conv_300m / hospital /	clinic_300m

In [ ]:
# 透過相似矩陣刪除: 'Unnamed: 0','bathroom','sale price','building completion year','lat','lng','transaction date'
dataset = pd.read_csv('/content/drive/MyDrive/極端值處理及常態分佈圖/ML data_outliers deletion_IQR.csv')
dataset['transaction month'] = dataset['transaction date'].str[5:7]
dataset['transaction month'] = dataset['transaction month'].astype(int)
dataset = dataset.drop(['Unnamed: 0','bathroom','sale price','building completion year','lat','lng','transaction date'] , axis=1)

# 新增生活機能*3: "conv distance_300m","hospital distance","clinic distance_300m"
dataset_new = pd.read_csv('/content/drive/MyDrive/ML/ML data_1019.csv')
dataset[["conv_300m","hospital","clinic_300m"]]= dataset_new[["conv distance_300m","hospital distance","clinic distance_300m"]]

# 把MRT/park的屬性改成作分類
# dataset["MRT"] = dataset["MRT_1km"] + dataset["MRT_500m"] + dataset["MRT_300m"]
# dataset["Park"] = dataset["park_500m"] + dataset["park_300m"]
# dataset = dataset.drop(["MRT_1km","MRT_500m","MRT_300m","park_500m","park_300m"], axis=1)

print(dataset.shape)
dataset.head()

(508431, 35)


,quater,main building area,transaction obj,building state,hall,room,bathroom,partition,mgmt org,sale price,berth total price,district,auxiliary building area,balcony area,elevator,building completion year,property age,unit price_ping,city,lat,lng,MRT_1km,MRT_500m,MRT_300m,park_500m,park_300m,elite_school,land shifting area_ping,building shifting area_ping,berth shifting area_ping,transaction year,transaction month,conv_300m,hospital,clinic_300m
0,1,49.09,1,0,2,2,1,1,1,13550000.0,0,18,0.00,6.07,0,2000,12,483131,1,24.985349,121.562827,0,0,0,1,1,0,54.214876,306.479339,57.685950,2012,10,10,0,15
1,1,113.67,0,4,2,4,2,1,1,17700000.0,0,18,4.00,13.29,0,1997,15,357058,1,24.983936,121.578889,0,0,0,1,0,0,174.082645,541.719008,0.000000,2012,10,3,0,0
2,1,34.02,1,4,1,2,2,1,1,8010000.0,0,18,2.69,3.92,0,2009,3,424721,1,25.002320,121.549945,1,0,0,1,1,0,47.140496,206.115702,0.000000,2012,10,9,1,11
3,1,27.68,0,3,1,1,1,1,0,7800000.0,0,18,0.34,2.66,0,1997,15,572436,1,25.002684,121.537224,1,1,1,1,1,1,35.305785,148.892562,0.000000,2012,10,11,0,12
4,1,68.17,1,4,2,3,2,1,1,22860000.0,1500000,18,10.97,7.12,0,2012,0,500010,1,25.005495,121.542196,1,1,0,1,1,1,102.148760,499.900826,33.057851,2012,10,3,0,2


## 眾多模型比較

### 相似矩陣刪除高相關屬性後

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import os
from google.colab import drive
#main libraries
import os
import re
import pickle
import numpy as np
import pandas as pd

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
import plotly.figure_factory as ff 
from plotly import tools

#machine learning libraries:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split, KFold, cross_val_score
from sklearn.preprocessing  import StandardScaler, LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import (BaggingRegressor, AdaBoostRegressor,GradientBoostingRegressor, 
                              RandomForestRegressor,  GradientBoostingRegressor)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score
from mlxtend.regressor import StackingCVRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor


In [ ]:
# data = dataset[:100]
y = dataset['unit price_ping']
X = dataset.drop(['unit price_ping'], axis=1)

In [ ]:
# define models to test:
# SVM": make_pipeline(RobustScaler(), SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, kernel='rbf', max_iter=-1, shrinking=True,
  # tol=0.001, verbose=False)),
base_models = {  "Lasso" : make_pipeline(RobustScaler(), Lasso(alpha =0.0005,random_state=1)),
    "Random Forest": RandomForestRegressor(n_estimators=1200,
                          max_depth=8,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42),
        
        "XGBoost": XGBRegressor(learning_rate=0.0001,objective ='reg:squarederror',
                       n_estimators=6000,
                       max_depth=15,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)}

In [ ]:
# Preprocessing, fitting, making predictions and scoring for every model:
train_set = X
Y_train = y
models_data = {'R^2':{'Training':{},'Testing':{}},
               'Adjusted R^2':{'Training':{},'Testing':{}},
               'MAE':{'Training':{},'Testing':{}},
               'MSE':{'Training':{},'Testing':{}},
               'RMSE':{'Training':{},'Testing':{}}}

X_train, X_test, y_train, y_test = train_test_split(train_set, Y_train, test_size=0.3, random_state=42)
p = train_set.shape[1]
train_n = X_train.shape[0]
test_n = X_test.shape[0]

for name in base_models:
    #fitting the model
    model = base_models[name].fit(X_train, y_train)
    #make predictions with train and test datasets
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    #calculate the R-Squared for training and testing
    r2_train,r2_test = model.score(X_train, y_train), model.score(X_test, y_test)
    models_data['R^2']['Training'][name], models_data['R^2']['Testing'][name] = r2_train, r2_test
            
    #calculate the Adjusted R-Squared for training and testing
    adj_train, adj_test = (1-(1-r2_train)*(train_n-1)/(train_n-p-1)) ,(1-(1-r2_test)*(train_n-1)/(train_n-p-1))
    models_data['Adjusted R^2']['Training'][name], models_data['Adjusted R^2']['Testing'][name] = adj_train, adj_test
               
    #calculate the Mean absolute error for training and testing
    mae_train, mae_test = mean_absolute_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)         
    models_data['MAE']['Training'][name], models_data['MAE']['Testing'][name] = mae_train, mae_test
               
    #calculate Mean square error for training and testing
    mse_train, mse_test = mean_squared_error(y_train, y_pred_train), mean_squared_error(y_test, y_pred_test)
    models_data['MSE']['Training'][name], models_data['MSE']['Testing'][name] = mse_train, mse_test

    #calculate Root mean error for training and testing    
    rmse_train, rmse_test = np.sqrt(mse_train), np.sqrt(mse_test)
    models_data['RMSE']['Training'][name], models_data['RMSE']['Testing'][name] = rmse_train, rmse_test
    
    print('\n========================={}========================='.format(name))
    print('**********Training**********************Testing********')
    print('R^2    : ',r2_train,' '*(25-len(str(r2_train))),r2_test) 
    print('Adj R^2: ',adj_train,' '*(25-len(str(adj_train))),adj_test) 
    print('MAE    : ',mae_train,' '*(25-len(str(mae_train))),mae_test) 
    print('MSE    : ',mse_train,' '*(25-len(str(mse_train))),mse_test) 
    print('RMSE   : ',rmse_train,' '*(25-len(str(rmse_train))),rmse_test)


=========================Lasso=========================
**********Training**********************Testing********
R^2    :  0.6092159839469979         0.6148144590957046
Adj R^2:  0.6091874334363752         0.6147863176072466
MAE    :  87997.07461796839          15036767171.94124
MSE    :  15209387015.145699         15036767171.94124
RMSE   :  123326.34355702637         122624.49662258044

=========================Random Forest=========================
**********Training**********************Testing********
R^2    :  0.7339152044264048         0.7327027671642043
Adj R^2:  0.733895764386714          0.7326832385443733
MAE    :  70616.49116163641          10434675835.494751
MSE    :  10356070024.562735         10434675835.494751
RMSE   :  101764.77791732627         102150.26106425158


In [ ]:
R_2 = pd.DataFrame(models_data['R^2']).sort_values(by='Testing',ascending=False)
Adjusted_R_2 = pd.DataFrame(models_data['Adjusted R^2']).sort_values(by='Testing',ascending=False)
MAE = pd.DataFrame(models_data['MAE']).sort_values(by='Testing',ascending=True)
MSE = pd.DataFrame(models_data['MSE']).sort_values(by='Testing',ascending=True)
RMSE = pd.DataFrame(models_data['RMSE']).sort_values(by='Testing',ascending=True)

In [ ]:
!pip install chart_studio
#visualizing the variance of R-Squared for each model
from  plotly.offline import plot
import chart_studio.plotly as py
# init_notebook_mode()
init_notebook_mode(connected=True)

#R2_包含training & testing包含training & testing
scores = pd.DataFrame(models_data['R^2']).sort_values(by='Testing',ascending=False)
# scores.plot(kind='bar',title='R-Squared for each model')
scores.plot(kind='bar',title='R-Squared for each model')
plt.title('R-Squared for each model', size=18)
plt.xlabel('Model', size=15, labelpad=10)
plt.ylabel('R-squared', size=15, labelpad=10)
plt.xticks(rotation=360)
# plt.savefig('/content/drive/MyDrive/ML/圖片/眾多模型比較/R_Squared for each model_include train_no sales price_V2.png',bbox_inches = 'tight',dpi=1000)

#RMSE
scores = pd.DataFrame(models_data['RMSE']).sort_values(by='Testing',ascending=True)
scores.plot(kind='bar',title='RMSE for each model')
plt.title('RMSE for each model', size=18)
plt.xlabel('Model', size=15, labelpad=10)
plt.ylabel('Unit price/ping', size=15, labelpad=10)
plt.xticks(rotation=360)
# plt.savefig('/content/drive/MyDrive/ML/圖片/眾多模型比較/RMSE for each model_no sales price_V2.png',bbox_inches = 'tight',dpi=1000)

# XGBoost模型畫圖

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#dataset = dataset[:300]
y = dataset['unit price_ping']
X = dataset.drop(['unit price_ping'], axis=1)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=0)
xgb_model = xgb.XGBRegressor(learning_rate=0.01,objective ='reg:squarederror',
                       n_estimators=6000,
                       max_depth=10,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=30,
                       reg_alpha=0.00006,
                       random_state=42)
xgb_model.fit(train_X, train_y)
# xgb_model.fit(train_X, train_y, eval_set=[(train_X, train_y)], early_stopping_rounds=10, verbose=False)

def print_evaluate(true, predicted):  
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
pred_y = xgb_model.predict(test_X)
pred_train_y = xgb_model.predict(train_X)
print('Test set evaluation:')
print_evaluate(pred_y, test_y)
print('Train set evaluation:')
print_evaluate(pred_train_y, train_y)

Test set evaluation:
MAE: 41891.08500097829
MSE: 4790417720.392076
RMSE: 69212.84360862567
R2 Square 0.8561348381415952
__________________________________
Train set evaluation:
MAE: 28282.942944153263
MSE: 1826478636.8123977
RMSE: 42737.32135747861
R2 Square 0.94763255698671
__________________________________


## XGBoost 準確率

### XGBoost feature importance - 以gain為指標

In [ ]:
from xgboost import plot_importance  
from matplotlib import pyplot 

ax = plot_importance(xgb_model,importance_type = 'gain',max_num_features=10,height=0.5,show_values = False,color='red')   
ax.figure.set_size_inches(7,5)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
ax.title.set_size(25)
ax.xaxis.label.set_size(20)
ax.yaxis.label.set_size(20)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_gain_red.png',bbox_inches = 'tight',dpi=700) 
pyplot.show()

### 利用0.7/0.3 -y VS yhat 畫圖

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from yellowbrick.regressor import residuals_plot
from yellowbrick.regressor import prediction_error
import yellowbrick
import seaborn as sns
import matplotlib.pyplot as plt

"""
# 切分資料輸入：特徵 輸出：預測目標變數
y = dataset['unit price_ping']
X = dataset.drop(['unit price_ping','Unnamed: 0','sale price'], axis=1)

# 切分訓練集、測試集,切分比例7:3
# 呼叫XGBoost模型，使用訓練集資料進行訓練（擬合）
# 使用模型對測試集資料進行預測
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=0)
xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate = 0.05, n_jobs=4)
xgb_model.fit(train_X, train_y, eval_set=[(train_X, train_y)], early_stopping_rounds=10, verbose=False)
pred_y = xgb_model.predict(test_X)
pred_train_y = xgb_model.predict(train_X)
"""
def scatterplot(x,y,x_line,y_line):
  fig, ax = plt.subplots(figsize=(50, 25))
  ax.scatter(x, y)
  # 設定刻度字型大小
  plt.xticks(fontsize=30)
  # plt.xticks(fontsize=30)
  plt.yticks(fontsize=30)
  # 設定xy軸大小 
  tick_x = [0,200000,400000,600000,800000,1000000,1200000,1400000,1600000]
  values_x = ['0K', '200K', '400K', '600K','800K','1000K','1200K','1400K','1600K'] 
  plt.xticks(tick_x, values_x)
  plt.yticks(tick_x, values_x)
  #adds a title and axes labels
  ax.set_title('XGBoost House Price Result',fontsize=40)
  ax.set_xlabel('Real Price ($)',fontsize=40)
  ax.set_ylabel('Prediction Price($)',fontsize=40)
 
  #removing top and right borders
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  # 畫直線
  
  plt.plot([0,1600000],[0,1600000],'r',linewidth=4.0)
  # plt.plot(x_line, y_line, 'r')
  plt.legend(("Pred","Ground Truth"),prop={'size': 40})
  plt.savefig('/content/drive/MyDrive/ML/圖片/House_RES_V2.png')  
  plt.show()

x = test_y
y = pred_y
x_line = max(y)
y_line = x_line

scatterplot(x,y,x_line,y_line)


### XGBoost 根據時間看準確率

#### Dataset + predict

In [ ]:
# 將predict的資訊跟原本data結合
#test
data_test = test_X
data_test['predict test']=0
for i in range(len(pred_y)):
  data_test.iloc[i,-1] = pred_y[i]
data_test['test_Y']=0
for i in range(len(test_y)):
  data_test.iloc[i,-1] = test_y.iloc[[i][-1]]

data_test.reset_index(inplace = True)
data_test.to_csv('/content/drive/MyDrive/ML/ML data_predict_test_v2.csv')

#train
data_train = train_X
data_train['predict train']=0
for i in range(len(pred_train_y)):
  data_train.iloc[i,-1] = pred_train_y[i]
data_train['train_Y']=0
for i in range(len(train_y)):
  data_train.iloc[i,-1] = train_y.iloc[[i][-1]]
data_train.reset_index(inplace = True)
data_train.to_csv('/content/drive/MyDrive/ML/ML data_predict_train_v2.csv')

# 使用 concat 合併 axis=0 為直向合併
data_new = pd.concat([data_test,data_train],axis=0)
data_new.reset_index(inplace = True)
data_new.to_csv('/content/drive/MyDrive/ML/ML data_predict_v2.csv')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML/ML data_predict_train_v2.csv')
# 先处理月份
df_m_s = data[data['transaction month'] < 10].copy()
df_m_b = data[data['transaction month'] >= 10].copy()
df_m_s['YearMonth'] = df_m_s['transaction year'].astype(str) + ('0' + df_m_s['transaction month'].astype(str))
df_m_b['YearMonth'] = df_m_b['transaction year'].astype(str) + df_m_b['transaction month'].astype(str)
data_train = pd.concat([df_m_s, df_m_b])
data_train.rename(columns={'predict': 'predict_train', 'test_Y': 'train_Y', 'YearMonth': 'YearMonth_train'}, inplace=True)
data_train

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ML/ML data_predict_test.csv')
# 先处理月份
df_m_s = data[data['transaction month'] < 10].copy()
df_m_b = data[data['transaction month'] >= 10].copy()
df_m_s['YearMonth'] = df_m_s['transaction year'].astype(str) + ('0' + df_m_s['transaction month'].astype(str))
df_m_b['YearMonth'] = df_m_b['transaction year'].astype(str) + df_m_b['transaction month'].astype(str)
data_test = pd.concat([df_m_s, df_m_b])
data_test.rename(columns={'predict test': 'predict_test', 'YearMonth': 'YearMonth_test'}, inplace=True)
data_test

#### 畫圖 for testing data

In [ ]:
# predict_test
import matplotlib.pyplot as plt 
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
t = data_test[['YearMonth_test','predict_test']]
y = t.groupby('YearMonth_test').mean()[['predict_test']]
y['YearMonth_test'] = y.index
y = y.reset_index(drop=True)
y['YearMonth_test'] = y['YearMonth_test'].apply(lambda _: datetime.strptime(_,"%Y%m"))
y.set_index('YearMonth_test', inplace=True)

print(y.dtypes)
y = y[12:]
z = y
z

In [ ]:
# test_Y
import matplotlib.pyplot as plt 
import pandas as pd
from pandas import DataFrame
from datetime import datetime, timedelta
t = data_test[['YearMonth_test','test_Y']]
y = t.groupby('YearMonth_test').mean()[['test_Y']]
y['YearMonth_test'] = y.index
y = y.reset_index(drop=True)
y['YearMonth_test'] = y['YearMonth_test'].apply(lambda _: datetime.strptime(_,"%Y%m"))
y.set_index('YearMonth_test', inplace=True)

print(y.dtypes)
y = y[12:]
y

In [ ]:
price_test = y.merge(z, how='inner', on='YearMonth_test')
price_test.rename(columns={'predict_test': 'Predict', 'test_Y': 'Ground truth'}, inplace=True)
price_test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

fig = plt.figure(figsize=(13,7), dpi=100)
ax = fig.add_subplot(111)

ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

ax.plot(price_test.index,price_test['Predict'], label='Predict',linewidth=5)
ax.plot(price_test.index,price_test['Ground truth'], label='Ground Truth',linewidth=4)

plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Time', fontsize=20)  
plt.ylabel("Unit price_ping", fontsize=20)    
ax.legend()                       
plt.title("XGBoost Price Predict", fontsize=25, color='black', pad=20)
plt.gcf().autofmt_xdate()

plt.savefig('/content/drive/MyDrive/ML/圖片/XGBoost Price Predict.png') 
plt.show()

## SHAP

In [ ]:
! pip install shap

In [ ]:
! pip install shap
import shap
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(train_X)
print(shap_values.shape)

### shap.summary_plot

In [ ]:
shap.summary_plot(shap_values, train_X,show=False)
plt.savefig('/content/drive/MyDrive/ML/圖片/shap value_V2.png',bbox_inches = 'tight') 

### 單一特徵 VS shap value

In [ ]:
for i in range(dataset.shape[1]):
  if (list(dataset)[i] != 'unit price_ping'):
    shap.dependence_plot(list(dataset)[i], shap_values, train_X,interaction_index=None, show=False)
    path = '/content/drive/MyDrive/ML/圖片/SHAP/單一特徵'
    # name = path+'2017-'+list(dataset)[i]+'.png'
    # plt.savefig('/content/drive/MyDrive/ML/圖片/shap_city.png',bbox_inches = 'tight')
    file_path = os.path.join(path, f"shap_{list(dataset)[i]}.png")
    plt.savefig(file_path ,bbox_inches = 'tight',dpi=700)

### 兩個特徵的交互作用 VS shap value

In [ ]:
for i in range(dataset.shape[1]):
  if (list(dataset)[i] != 'unit price_ping'):
    shap.dependence_plot(list(dataset)[i], shap_values, train_X , show=False)
    path = '/content/drive/MyDrive/ML/圖片/SHAP/兩特徵交互作用'
    file_path = os.path.join(path, f"shap_{list(dataset)[i]}.png")
    plt.savefig(file_path ,bbox_inches = 'tight',dpi=700)

## XGBoost - feature importance

In [ ]:
from xgboost import plot_importance  
from matplotlib import pyplot 
# plt.figure(figsize=(100,80))
ax = plot_importance(xgb_model,importance_type = 'cover')  
ax.figure.set_size_inches(30,15)

# 設定刻度字型大小
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
#adds a title and axes labels
ax.set_title(fontsize=55)
ax.set_xlabel(fontsize=43)
ax.set_ylabel(fontsize=43)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_cover_V2.png',bbox_inches = 'tight',dpi=700)
pyplot.show()

#預設
ax = plot_importance(xgb_model,importance_type = 'weight') 
ax.figure.set_size_inches(20,10)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_weight_V2.png',bbox_inches = 'tight',dpi=700)
pyplot.show()

ax = plot_importance(xgb_model,importance_type = 'gain')  
ax.figure.set_size_inches(20,10)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_gain_V2.png',bbox_inches = 'tight',dpi=700)
pyplot.show()

ax = plot_importance(xgb_model,importance_type = 'total_gain')  
ax.figure.set_size_inches(20,10)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_total_gain_V2.png',bbox_inches = 'tight',dpi=700)
pyplot.show()

ax = plot_importance(xgb_model,importance_type = 'total_cover')  
ax.figure.set_size_inches(20,10)
plt.savefig('/content/drive/MyDrive/ML/圖片/xgboost_feature_total_cover_V2.png',bbox_inches = 'tight',dpi=700)
pyplot.show()